In [1]:
# Get all the players from Sleeper and store the pertinent information in a SQLite database
import requests
import sqlite3
import json
from sleeper import get_all_players, get_users_in_league, get_all_rosters
from manage import clean_player_data
from config import LEAGUE_LIST
import pandas as pd
LEAGUE_ID = "1234567890123456789"
SLEEPER_API = "https://api.sleeper.app/v1/players/nfl"


In [2]:
# Get all the players from Sleeper
players_json = get_all_players()
players_df = clean_player_data(players_json)

# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
players_df.to_sql('players', conn, if_exists='replace', index=True)
conn.commit()
conn.close()

In [3]:
players_df['team'].unique()

array(['FA', 'TB', 'PIT', 'JAX', 'ATL', 'MIA', 'CHI', 'DAL', 'PHI', 'CLE',
       'SF', 'NYG', 'NE', 'DEN', 'LAR', 'BAL', 'MIN', 'IND', 'CAR', 'LAC',
       'WAS', 'HOU', 'LV', 'BUF', 'KC', 'NYJ', 'CIN', 'GB', 'ARI', 'DET',
       'SEA', 'TEN', 'NO'], dtype=object)

# Insert the league id with the league name into the db
This is a manual process of keeping this up to date

In [3]:
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Remove the coaches table if it exists already
conn.execute("DROP TABLE IF EXISTS league")

# Convert to a DataFrame
df = pd.DataFrame(list(LEAGUE_LIST.items()), columns=['name', 'league_id'])
df.to_sql('league', conn, if_exists='append', index=False)


8

# Insert the coaches into the database

In [4]:
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Remove the coaches table if it exists already
conn.execute("DROP TABLE IF EXISTS coaches")
# Get all the coaches in each of the leagues
for league in LEAGUE_LIST:
    coaches_df = get_users_in_league(LEAGUE_LIST[league])
    coaches_df.to_sql('coaches', conn, if_exists='append', index=False)

# Update the team names if they aren't already set by combining the display name and Team like "Team" + Display Name from the row
conn.execute("UPDATE coaches SET team_name = 'Team ' + display_name WHERE team_name IS NULL")

# Close the connection
conn.close()

# Now get the drafts from every league and put these in the database

In [5]:
# Get the drafts for each league
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
conn.execute("DROP TABLE IF EXISTS rosters")
for league in LEAGUE_LIST:
    rosters = get_all_rosters(LEAGUE_LIST[league])
    # Expand the players columns into individual rows keeping just the player_id, the owner_id, and league_id
    rosters_df = rosters.explode('players')[['players','owner_id','league_id']].rename(columns={'players':'player_id'})
    duplicates = rosters_df.duplicated()
    print(f"Number of duplicate rows: {duplicates.sum()}")
    
    rosters_df.to_sql('rosters', conn, if_exists='append', index=False)
# Close the connection
conn.close()


Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0
Number of duplicate rows: 0


# Load Bye Weeks into the Database

In [6]:
# Load bye weeks into the SQLite database
conn = sqlite3.connect('../data/league_db.db')
conn.execute("DROP TABLE IF EXISTS team")
# Open the file with the bye weeks
with open('../data/nfl_bye_weeks_2024.csv', 'r') as file:
    # Read the data into a dataframe
    bye_weeks = pd.read_csv(file)
    # Insert the data into the database
    bye_weeks.to_sql('team', conn, if_exists='replace', index=False)
conn.close()

erDiagram
    PLAYERS {
        int player_id PK
        int rotoworld_id
        float weight
        string practice_description
        date birth_date
        string team_abbr
        int fantasy_data_id
        bool active
        int opta_id
        int age
        int espn_id
        string first_name
        string hashtag
        int depth_chart_order
        string status
        int pandascore_id
        int stats_id
        int swish_id
        string search_first_name
        string search_last_name
        int oddsjam_id
        string college
        string last_name
        string team
        string full_name
        string search_full_name
        float height
        string fantasy_positions
        string practice_participation
        string position
        int years_exp
        int number
        string gsis_id
        string depth_chart_position
        int search_rank
        string sport
    }

    LEAGUE {
        int league_id PK
        string name
    }

    COACHES {
        int user_id PK
        string display_name
        bool is_owner
        int league_id FK
        string team_name
        date team_name_update
        bool allow_sms
    }

    ROSTERS {
        int roster_id PK
        int player_id FK
        int owner_id FK
        int league_id FK
    }

    TEAM {
        string Team PK
        int Bye_Week
    }

    PLAYERS ||--o{ ROSTERS : "is part of"
    COACHES ||--o{ ROSTERS : "owns"
    LEAGUE ||--o{ ROSTERS : "includes"
    LEAGUE ||--o{ COACHES : "has"
    TEAM ||--o{ PLAYERS : "belongs to"


# Create spreadsheet of rostered players

In [8]:
conn = sqlite3.connect('../data/league_db.db')
rows = conn.execute(f'Select p.full_name, p.espn_id,p.rotowire_id, p.sportradar_id,p.yahoo_id, p.college, p.years_exp, p.team, p.position, c.display_name, c.league_id, t."Bye Week"'
                    'FROM players p '
                    'JOIN rosters r on p.player_id = r.player_id '
                    'JOIN coaches c on r.owner_id = c.user_id and r.league_id = c.league_id '
                    'JOIN team t on p.team=t.team').fetchall()
# turn the rows into a dataframe
df = pd.DataFrame(rows, columns=['full_name','espn_id','rotowire_id', 'sportradar_id','yahoo_id','college', 'years_experience','team', 'position', 'coach','league_id','bye'])
conn.close()

# Create a reverse dictionary to map IDs to names
reverse_LEAGUE_LIST = {str(v): k for k, v in LEAGUE_LIST.items()}
# Ensure the league_id column is of type string
df['league_id'] = df['league_id'].astype(str)
df['espn_id']=df['espn_id'].fillna(-1).astype(int)
df['rotowire_id']=df['rotowire_id'].fillna(-1).astype(int)
df['yahoo_id']=df['yahoo_id'].fillna(-1).astype(int)

# Now replace the league name with the league id
df['league_name'] = df['league_id'].map(reverse_LEAGUE_LIST)
df.drop(columns=['league_id'], inplace=True)

# Output the data to a csv file
df.to_csv('../data/players_rostered.csv', index=False)
df

,full_name,espn_id,rotowire_id,sportradar_id,yahoo_id,college,years_experience,team,position,coach,bye,league_name
0,Jonathan Mingo,-1,16937,6cf0c82d-87e4-4cfc-8475-77667ed92796,-1,Ole Miss,1.0,CAR,WR,ChiJV,11,MainLeague
1,Rashee Rice,-1,16963,cbd73c09-da07-48b0-a0d6-f248dfa631b6,-1,SMU,1.0,KC,WR,ChiJV,6,MainLeague
2,Roschon Johnson,-1,16772,e3c414c0-be8c-4807-af9c-f91d89c83e60,-1,Texas,1.0,CHI,RB,ChiJV,7,MainLeague
3,Brenden Rice,-1,17756,5c33558d-02fb-4c99-88d3-37083472c19f,-1,USC,0.0,LAC,WR,ChiJV,5,MainLeague
4,Luke McCaffrey,-1,17820,5cfd82f8-9e73-48d7-914e-9051ff52d67d,-1,Rice,0.0,WAS,WR,ChiJV,14,MainLeague
...,...,...,...,...,...,...,...,...,...,...,...,...
2021,Kenneth Walker,-1,15909,22ee9bac-a64c-4d44-94fc-51d775465b3b,-1,Michigan State,2.0,SEA,RB,WestCoastNonsense,10,Quickdraft 2
2022,Brock Purdy,-1,15834,d5aef708-ad61-4ab8-a637-62ff96e92040,-1,Iowa State,2.0,SF,QB,WestCoastNonsense,9,Quickdraft 2
2023,Anthony Richardson,-1,16653,5fc5b4a6-a583-4345-a810-4f982204ea5e,-1,Florida,1.0,IND,QB,WestCoastNonsense,14,Quickdraft 2
2024,C.J. Stroud,-1,16886,7e91086f-9ca4-4bd9-a099-b88b2f37e6c6,-1,Ohio State,1.0,HOU,QB,WestCoastNonsense,14,Quickdraft 2


In [15]:
df[df.team=="FA"]

,full_name,college,years_experience,team,position,coach,bye,league_name
166,Michael Thomas,Ohio State,8.0,FA,WR,tford51,0,MainLeague
287,Jerick McKinnon,Georgia Southern,10.0,FA,RB,CoryPeisert,0,MainLeague
416,JuJu Smith-Schuster,USC,7.0,FA,WR,theBandit216,0,BB1
516,Michael Gallup,Colorado State,6.0,FA,WR,ConorDeLa,0,BB1
537,Michael Thomas,Ohio State,8.0,FA,WR,ChiJV,0,BB1
587,Darren Waller,Georgia Tech,9.0,FA,TE,plfinken,0,BB1
609,Dalvin Cook,Florida State,7.0,FA,RB,aledo,0,BB1
766,Michael Gallup,Colorado State,6.0,FA,WR,ConorDeLa,0,BB2
835,Darren Waller,Georgia Tech,9.0,FA,TE,Deerslayer65,0,BB2
886,Michael Thomas,Ohio State,8.0,FA,WR,ChiJV,0,BB2


In [10]:
# Run a sqlite query
def run_query(q):
    conn = sqlite3.connect('../data/league_db.db')
    rows = conn.execute(q).fetchall()
    df = pd.DataFrame(rows)
    return df

def get_column_names(table_name):
    # Connect to the SQLite3 database
    conn = sqlite3.connect('../data/league_db.db')
    cursor = conn.cursor()
    
    # Execute PRAGMA table_info to get the column information
    cursor.execute(f"PRAGMA table_info({table_name})")
    
    # Fetch all results
    columns_info = cursor.fetchall()
    
    # Extract column names
    column_names = [info[1] for info in columns_info]
    
    # Close the connection
    conn.close()
    
    return column_names

def table_names():
    # Get the list of tables in teh sqlite database
    conn = sqlite3.connect('../data/league_db.db')
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    return tables


for table in table_names():
    print(f'{table[0]}: {get_column_names(table[0])}')
    
#run_query('Select p.full_name, r.league_id, p.player_id '
                    #'FROM players p join rosters r on r.player_id = p.player_id WHERE full_name like "%Davante Adams%"')


players: ['player_id', 'fantasy_data_id', 'rotoworld_id', 'swish_id', 'stats_id', 'active', 'hashtag', 'depth_chart_position', 'full_name', 'height', 'practice_description', 'practice_participation', 'first_name', 'status', 'opta_id', 'oddsjam_id', 'fantasy_positions', 'espn_id', 'search_last_name', 'college', 'sport', 'search_rank', 'number', 'depth_chart_order', 'age', 'gsis_id', 'search_first_name', 'weight', 'last_name', 'birth_date', 'years_exp', 'position', 'team', 'team_abbr', 'search_full_name', 'pandascore_id']
league: ['name', 'league_id']
coaches: ['display_name', 'is_owner', 'league_id', 'user_id', 'team_name', 'team_name_update', 'allow_sms']
rosters: ['player_id', 'owner_id', 'league_id']
team: ['Team', 'Bye Week']


In [27]:
run_query('Select l.name, c.display_name, count(r.player_id) from rosters r, coaches c, league l where r.owner_id = c.user_id and l.league_id=r.league_id group by l.name, c.display_name order by l.name, c.display_name')

,0,1,2
0,BB1,ChiJV,75
1,BB1,ConorDeLa,200
2,BB1,Mschmoopie,150
3,BB1,ProfessorD,200
4,BB1,WestCoastNonsense,200
...,...,...,...
83,Quickdraft 2,WestCoastNonsense,128
84,Quickdraft 2,aledo,128
85,Quickdraft 2,koolaid403,128
86,Quickdraft 2,tford51,128


In [14]:
#run_query('select team, * from players where full_name like "%Dalvin Cook%"')
run_query('select espn_id from players').count()

0    2210
dtype: int64